# Joins/Merges, Grouping, and Windowing with DuckDB

In medical data integration, we often need to combine data from multiple sources, aggregate information, and perform complex analytical queries. This notebook introduces SQL operations using DuckDB for handling medical datasets efficiently.

First, let's install and import the necessary libraries for our medical data analysis.

Now we'll import the required libraries and set up our DuckDB connection.

In [1]:
import duckdb
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Create DuckDB connection
conn = duckdb.connect(':memory:')

Let's create sample medical datasets that simulate real-world scenarios: patient information, lab results, and medication records.

In [2]:
# Create patients table
patients = pd.DataFrame({
    'patient_id': [1, 2, 3, 4, 5],
    'age': [45, 32, 67, 23, 78],
    'gender': ['M', 'F', 'M', 'F', 'M'],
    'diagnosis': ['Hypertension', 'Diabetes', 'Heart Disease', 'Diabetes', 'Hypertension']
})

print("Patients Table:")
print(patients)

Patients Table:
   patient_id  age gender      diagnosis
0           1   45      M   Hypertension
1           2   32      F       Diabetes
2           3   67      M  Heart Disease
3           4   23      F       Diabetes
4           5   78      M   Hypertension


Next, we'll create a lab results table with multiple measurements per patient over time.

In [3]:
# Create lab results table
lab_results = pd.DataFrame({
    'patient_id': [1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 1, 2],
    'test_date': ['2024-01-15', '2024-02-15', '2024-01-10', '2024-02-10', 
                  '2024-01-20', '2024-02-20', '2024-01-25', '2024-02-25',
                  '2024-01-30', '2024-02-28', '2024-03-15', '2024-03-10'],
    'test_type': ['Blood Glucose', 'Blood Pressure', 'Blood Glucose', 'HbA1c',
                  'Cholesterol', 'Blood Pressure', 'Blood Glucose', 'HbA1c',
                  'Blood Pressure', 'Cholesterol', 'Blood Glucose', 'Blood Glucose'],
    'result_value': [120, 140, 180, 7.2, 220, 160, 95, 6.8, 150, 200, 110, 170]
})

print("Lab Results Table:")
print(lab_results)

Lab Results Table:
    patient_id   test_date       test_type  result_value
0            1  2024-01-15   Blood Glucose         120.0
1            1  2024-02-15  Blood Pressure         140.0
2            2  2024-01-10   Blood Glucose         180.0
3            2  2024-02-10           HbA1c           7.2
4            3  2024-01-20     Cholesterol         220.0
5            3  2024-02-20  Blood Pressure         160.0
6            4  2024-01-25   Blood Glucose          95.0
7            4  2024-02-25           HbA1c           6.8
8            5  2024-01-30  Blood Pressure         150.0
9            5  2024-02-28     Cholesterol         200.0
10           1  2024-03-15   Blood Glucose         110.0
11           2  2024-03-10   Blood Glucose         170.0


Now we'll create a medications table to demonstrate more complex join operations.

In [4]:
# Create medications table
medications = pd.DataFrame({
    'patient_id': [1, 2, 3, 4, 5, 2],
    'medication': ['Lisinopril', 'Metformin', 'Atorvastatin', 'Insulin', 'Amlodipine', 'Glipizide'],
    'start_date': ['2024-01-01', '2024-01-05', '2024-01-15', '2024-01-20', '2024-01-25', '2024-02-01'],
    'dosage': ['10mg', '500mg', '20mg', '10 units', '5mg', '5mg']
})

print("Medications Table:")
print(medications)

Medications Table:
   patient_id    medication  start_date    dosage
0           1    Lisinopril  2024-01-01      10mg
1           2     Metformin  2024-01-05     500mg
2           3  Atorvastatin  2024-01-15      20mg
3           4       Insulin  2024-01-20  10 units
4           5    Amlodipine  2024-01-25       5mg
5           2     Glipizide  2024-02-01       5mg


Let's register these DataFrames as tables in DuckDB so we can perform SQL queries on them.

In [5]:
# Register DataFrames as DuckDB tables
conn.register('patients', patients)
conn.register('lab_results', lab_results)
conn.register('medications', medications)

print("Tables registered successfully in DuckDB")

Tables registered successfully in DuckDB


## Inner Join Operations

Let's start with a basic inner join to combine patient information with their lab results.

In [6]:
# Inner join patients with lab results
query = """
SELECT p.patient_id, p.age, p.diagnosis, l.test_date, l.test_type, l.result_value
FROM patients p
INNER JOIN lab_results l ON p.patient_id = l.patient_id
ORDER BY p.patient_id, l.test_date
"""

result = conn.execute(query).fetchdf()
print("Inner Join - Patients with Lab Results:")
print(result)

Inner Join - Patients with Lab Results:
    patient_id  age      diagnosis   test_date       test_type  result_value
0            1   45   Hypertension  2024-01-15   Blood Glucose         120.0
1            1   45   Hypertension  2024-02-15  Blood Pressure         140.0
2            1   45   Hypertension  2024-03-15   Blood Glucose         110.0
3            2   32       Diabetes  2024-01-10   Blood Glucose         180.0
4            2   32       Diabetes  2024-02-10           HbA1c           7.2
5            2   32       Diabetes  2024-03-10   Blood Glucose         170.0
6            3   67  Heart Disease  2024-01-20     Cholesterol         220.0
7            3   67  Heart Disease  2024-02-20  Blood Pressure         160.0
8            4   23       Diabetes  2024-01-25   Blood Glucose          95.0
9            4   23       Diabetes  2024-02-25           HbA1c           6.8
10           5   78   Hypertension  2024-01-30  Blood Pressure         150.0
11           5   78   Hypertension  

Now let's perform a left join to include all patients, even those without lab results.

## Multiple Table Joins

Let's perform a three-way join to combine patients, lab results, and medications for comprehensive patient profiles.

In [7]:
# Three-way join for comprehensive patient view
query = """
SELECT DISTINCT p.patient_id, p.diagnosis, p.age,
       COUNT(DISTINCT l.test_type) as unique_tests,
       COUNT(DISTINCT m.medication) as num_medications
FROM patients p
LEFT JOIN lab_results l ON p.patient_id = l.patient_id
LEFT JOIN medications m ON p.patient_id = m.patient_id
GROUP BY p.patient_id, p.diagnosis, p.age
ORDER BY p.patient_id
"""

result = conn.execute(query).fetchdf()
print("Three-way Join - Complete Patient Profile:")
print(result)

Three-way Join - Complete Patient Profile:
   patient_id      diagnosis  age  unique_tests  num_medications
0           1   Hypertension   45             2                1
1           2       Diabetes   32             2                2
2           3  Heart Disease   67             2                1
3           4       Diabetes   23             2                1
4           5   Hypertension   78             2                1


## Grouping and Aggregation

Let's group patients by diagnosis and calculate summary statistics for each condition.

In [8]:
# Group by diagnosis with aggregations
query = """
SELECT p.diagnosis,
       COUNT(DISTINCT p.patient_id) as patient_count,
       AVG(p.age) as avg_age,
       MIN(p.age) as min_age,
       MAX(p.age) as max_age
FROM patients p
GROUP BY p.diagnosis
ORDER BY patient_count DESC
"""

result = conn.execute(query).fetchdf()
print("Grouping by Diagnosis:")
print(result)

Grouping by Diagnosis:
       diagnosis  patient_count  avg_age  min_age  max_age
0       Diabetes              2     27.5       23       32
1   Hypertension              2     61.5       45       78
2  Heart Disease              1     67.0       67       67


Now let's analyze lab results by test type with more detailed aggregations.

In [9]:
# Analyze lab results by test type
query = """
SELECT test_type,
       COUNT(*) as total_tests,
       COUNT(DISTINCT patient_id) as unique_patients,
       AVG(result_value) as avg_result,
       STDDEV(result_value) as std_result,
       MIN(result_value) as min_result,
       MAX(result_value) as max_result
FROM lab_results
GROUP BY test_type
ORDER BY total_tests DESC
"""

result = conn.execute(query).fetchdf()
print("Lab Results Analysis by Test Type:")
print(result)

Lab Results Analysis by Test Type:
        test_type  total_tests  unique_patients  avg_result  std_result  \
0   Blood Glucose            5                3       135.0   37.749172   
1  Blood Pressure            3                3       150.0   10.000000   
2           HbA1c            2                2         7.0    0.282843   
3     Cholesterol            2                2       210.0   14.142136   

   min_result  max_result  
0        95.0       180.0  
1       140.0       160.0  
2         6.8         7.2  
3       200.0       220.0  


## Window Functions

Window functions are powerful for analyzing trends over time. Let's track each patient's lab result trends using ROW_NUMBER and LAG functions.

In [10]:
# Use window functions to track lab result trends
query = """
SELECT patient_id, test_date, test_type, result_value,
       ROW_NUMBER() OVER (PARTITION BY patient_id, test_type ORDER BY test_date) as test_sequence,
       LAG(result_value) OVER (PARTITION BY patient_id, test_type ORDER BY test_date) as previous_result
FROM lab_results
WHERE test_type = 'Blood Glucose'
ORDER BY patient_id, test_date
"""

result = conn.execute(query).fetchdf()
print("Blood Glucose Trends with Window Functions:")
print(result)

Blood Glucose Trends with Window Functions:
   patient_id   test_date      test_type  result_value  test_sequence  \
0           1  2024-01-15  Blood Glucose         120.0              1   
1           1  2024-03-15  Blood Glucose         110.0              2   
2           2  2024-01-10  Blood Glucose         180.0              1   
3           2  2024-03-10  Blood Glucose         170.0              2   
4           4  2024-01-25  Blood Glucose          95.0              1   

   previous_result  
0              NaN  
1            120.0  
2              NaN  
3            180.0  
4              NaN  


Let's calculate the change in lab values between consecutive tests for each patient.

In [11]:
# Calculate changes between consecutive tests
query = """
WITH lab_changes AS (
    SELECT patient_id, test_date, test_type, result_value,
           LAG(result_value) OVER (PARTITION BY patient_id, test_type ORDER BY test_date) as previous_result,
           LAG(test_date) OVER (PARTITION BY patient_id, test_type ORDER BY test_date) as previous_date
    FROM lab_results
)
SELECT patient_id, test_type, test_date, result_value, previous_result,
       (result_value - previous_result) as value_change,
       CASE 
           WHEN previous_result IS NULL THEN 'First Test'
           WHEN result_value > previous_result THEN 'Increased'
           WHEN result_value < previous_result THEN 'Decreased'
           ELSE 'No Change'
       END as trend
FROM lab_changes
WHERE test_type = 'Blood Glucose'
ORDER BY patient_id, test_date
"""

result = conn.execute(query).fetchdf()
print("Blood Glucose Changes Over Time:")
print(result)

Blood Glucose Changes Over Time:
   patient_id      test_type   test_date  result_value  previous_result  \
0           1  Blood Glucose  2024-01-15         120.0              NaN   
1           1  Blood Glucose  2024-03-15         110.0            120.0   
2           2  Blood Glucose  2024-01-10         180.0              NaN   
3           2  Blood Glucose  2024-03-10         170.0            180.0   
4           4  Blood Glucose  2024-01-25          95.0              NaN   

   value_change       trend  
0           NaN  First Test  
1         -10.0   Decreased  
2           NaN  First Test  
3         -10.0   Decreased  
4           NaN  First Test  


Now let's use ranking functions to identify patients with the highest and lowest lab values.

In [12]:
# Ranking patients by lab results
query = """
SELECT patient_id, test_type, result_value,
       RANK() OVER (PARTITION BY test_type ORDER BY result_value DESC) as rank_high_to_low,
       PERCENT_RANK() OVER (PARTITION BY test_type ORDER BY result_value) as percentile_rank
FROM lab_results
WHERE test_type IN ('Blood Glucose', 'Blood Pressure')
ORDER BY test_type, rank_high_to_low
"""

result = conn.execute(query).fetchdf()
print("Patient Rankings by Lab Results:")
print(result)

Patient Rankings by Lab Results:
   patient_id       test_type  result_value  rank_high_to_low  percentile_rank
0           2   Blood Glucose         180.0                 1             1.00
1           2   Blood Glucose         170.0                 2             0.75
2           1   Blood Glucose         120.0                 3             0.50
3           1   Blood Glucose         110.0                 4             0.25
4           4   Blood Glucose          95.0                 5             0.00
5           3  Blood Pressure         160.0                 1             1.00
6           5  Blood Pressure         150.0                 2             0.50
7           1  Blood Pressure         140.0                 3             0.00


## Moving Averages and Running Totals

Let's calculate moving averages for lab results to smooth out fluctuations and identify trends.

In [13]:
# Calculate moving averages for lab results
query = """
SELECT patient_id, test_date, test_type, result_value,
       AVG(result_value) OVER (
           PARTITION BY patient_id, test_type 
           ORDER BY test_date 
           ROWS BETWEEN 1 PRECEDING AND CURRENT ROW
       ) as moving_avg_2_tests,
       AVG(result_value) OVER (
           PARTITION BY patient_id, test_type 
           ORDER BY test_date 
           ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
       ) as cumulative_avg
FROM lab_results
WHERE test_type = 'Blood Glucose'
ORDER BY patient_id, test_date
"""

result = conn.execute(query).fetchdf()
print("Moving Averages for Blood Glucose:")
print(result)

Moving Averages for Blood Glucose:
   patient_id   test_date      test_type  result_value  moving_avg_2_tests  \
0           1  2024-01-15  Blood Glucose         120.0               120.0   
1           1  2024-03-15  Blood Glucose         110.0               115.0   
2           2  2024-01-10  Blood Glucose         180.0               180.0   
3           2  2024-03-10  Blood Glucose         170.0               175.0   
4           4  2024-01-25  Blood Glucose          95.0                95.0   

   cumulative_avg  
0           120.0  
1           115.0  
2           180.0  
3           175.0  
4            95.0  


## Advanced Analytics: Patient Risk Scoring

Let's create a comprehensive query that combines multiple concepts to generate risk scores for patients.

In [14]:
# Complex query for patient risk assessment
query = """
WITH patient_metrics AS (
    SELECT p.patient_id, p.age, p.diagnosis,
           COUNT(l.test_date) as total_tests,
           AVG(CASE WHEN l.test_type = 'Blood Glucose' THEN l.result_value END) as avg_glucose,
           MAX(CASE WHEN l.test_type = 'Blood Pressure' THEN l.result_value END) as max_bp,
           COUNT(m.medication) as medication_count
    FROM patients p
    LEFT JOIN lab_results l ON p.patient_id = l.patient_id
    LEFT JOIN medications m ON p.patient_id = m.patient_id
    GROUP BY p.patient_id, p.age, p.diagnosis
)
SELECT patient_id, diagnosis, age,
       avg_glucose, max_bp, medication_count,
       -- Simple risk scoring based on multiple factors
       (CASE WHEN age > 65 THEN 2 ELSE 0 END +
        CASE WHEN avg_glucose > 140 THEN 3 ELSE 0 END +
        CASE WHEN max_bp > 140 THEN 2 ELSE 0 END +
        CASE WHEN medication_count > 1 THEN 1 ELSE 0 END) as risk_score,
       RANK() OVER (ORDER BY 
           (CASE WHEN age > 65 THEN 2 ELSE 0 END +
            CASE WHEN avg_glucose > 140 THEN 3 ELSE 0 END +
            CASE WHEN max_bp > 140 THEN 2 ELSE 0 END +
            CASE WHEN medication_count > 1 THEN 1 ELSE 0 END) DESC
       ) as risk_rank
FROM patient_metrics
ORDER BY risk_score DESC, patient_id
"""

result = conn.execute(query).fetchdf()
print("Patient Risk Assessment:")
print(result)

Patient Risk Assessment:
   patient_id      diagnosis  age  avg_glucose  max_bp  medication_count  \
0           3  Heart Disease   67          NaN   160.0                 2   
1           5   Hypertension   78          NaN   150.0                 2   
2           2       Diabetes   32        175.0     NaN                 6   
3           1   Hypertension   45        115.0   140.0                 3   
4           4       Diabetes   23         95.0     NaN                 2   

   risk_score  risk_rank  
0           5          1  
1           5          1  
2           4          3  
3           1          4  
4           1          4  


Finally, let's clean up our DuckDB connection.

In [15]:
# Close the DuckDB connection
conn.close()
print("DuckDB connection closed successfully")

DuckDB connection closed successfully


## Exercise

Create a comprehensive analysis using the concepts learned in this notebook:

1. **Data Setup**: Create three new tables - `hospitals` (hospital_id, name, location), `doctors` (doctor_id, name, specialty, hospital_id), and `appointments` (appointment_id, patient_id, doctor_id, appointment_date, diagnosis_code).

2. **Join Operations**: Write queries to:
   - Find all appointments with patient names, doctor names, and hospital information
   - Identify patients who have never had an appointment (use appropriate join)

3. **Grouping and Aggregation**: 
   - Calculate the number of appointments per hospital and specialty
   - Find the average age of patients by diagnosis

4. **Window Functions**:
   - Rank doctors by the number of appointments they have
   - For each patient, show their appointment history with sequence numbers
   - Calculate a 3-appointment moving average of patient ages for each doctor

5. **Advanced Analysis**: Create a query that identifies the top 3 busiest hospitals and shows trends in appointment volume over time using window functions.

Bonus: Implement a patient follow-up analysis that identifies patients who haven't had an appointment in the last 90 days but have chronic conditions requiring regular monitoring.